In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.events import EventFiringWebDriver, AbstractEventListener
from selenium.webdriver import Firefox
import time
from selenium.webdriver.common.action_chains import ActionChains

df=pd.DataFrame(columns=['route_name','route_link','bus_name','bus_type','departing_time','duration','reaching_time','star_rating','price','seats_available'])
df

,route_name,route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seats_available


In [6]:
travels_links_dict = {}

driver = webdriver.Firefox()
driver.get('https://www.redbus.in/')
driver.maximize_window()

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'rtcName')))
popular_bus_depts = driver.find_elements(By.CLASS_NAME, 'rtcName')
view_buses_buttons = driver.find_element(By.XPATH, "/html/body/section/div[2]/main/div[3]/div[3]/div[1]/div[2]/a")
popular_bus_names = [bus.text for bus in popular_bus_depts] 
driver.get(view_buses_buttons.get_attribute('href'))
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'D113_link')))
bus_depts = driver.find_elements(By.CLASS_NAME, 'D113_link')
d113_bus_dict={}
for bus in bus_depts:
    for name in popular_bus_names:
        if name in bus.text:
            d113_bus_dict[name]=bus.get_attribute('href')
driver.quit()

In [7]:
driver=webdriver.Firefox()
driver.maximize_window()
for name_string, link in d113_bus_dict.items():
    try:
        driver.get(link)
        time.sleep(2)
        
        # Wait for the route elements to be present
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "route"))
        )
        
        try:
            # Check if pagination exists
            parent_div = driver.find_element(By.CLASS_NAME, "DC_117_paginationTable")
            
            try:
                # Try to find pagination buttons
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.DC_117_pageTabs"))
                )
                
                child_count = driver.execute_script(
                    "return arguments[0].getElementsByTagName('div').length;", parent_div
                )
                
                # If buttons are found, iterate through pagination
                for i in range(1, child_count + 1):
                    button = driver.find_element(css selector"div.DC_117_pageTabs:nth-child({i})")
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
                    driver.execute_script("arguments[0].click();", button)
                    
                    # Extract route details on each page
                    route_details = WebDriverWait(driver, 5).until(
                        EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
                    )
                    for route in route_details:
                        route_link = route.get_attribute('href')
                        travels_links_dict[name_string + '_' + route.text] = route_link
                    
                    time.sleep(1)
                    
            except TimeoutException:
                # If buttons aren't found but parent div exists, get routes from current page
                print(f"Pagination buttons not found for link: {link}. Extracting from current page...")
                route_details = driver.find_elements(By.CLASS_NAME, 'route')
                for route in route_details:
                    route_link = route.get_attribute('href')
                    travels_links_dict[name_string + '_' + route.text] = route_link
                    
        except NoSuchElementException:
            # If pagination table doesn't exist, get routes from current page
            print(f"No pagination found for link: {link}. Extracting from current page...")
            route_details = driver.find_elements(By.CLASS_NAME, 'route')
            for route in route_details:
                route_link = route.get_attribute('href')
                travels_links_dict[name_string + '_' + route.text] = route_link
                
    except Exception as e:
        print(f"Error occurred while processing link: {link}. Skipping {name_string}")
        continue  # Skip to the next link
driver.quit()

Error occurred while processing link: https://www.redbus.in/online-booking/bsrtc-operated-by-vip-travels. Skipping... Exception: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Error occurred while processing link: https://www.redbus.in/online-booking/pepsu-punjab. Skipping... Exception: name 'NoSuchElementException' is not defined


In [8]:
travels_links_dict

{'APSRTC_Hyderabad to Vijayawada': 'https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada',
 'APSRTC_Vijayawada to Hyderabad': 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad',
 'APSRTC_Hyderabad to Ongole': 'https://www.redbus.in/bus-tickets/hyderabad-to-ongole',
 'APSRTC_Ongole to Hyderabad': 'https://www.redbus.in/bus-tickets/ongole-to-hyderabad',
 'APSRTC_Kakinada to Visakhapatnam': 'https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam',
 'APSRTC_Bangalore to Tirupati': 'https://www.redbus.in/bus-tickets/bangalore-to-tirupathi',
 'APSRTC_Bangalore to Kadapa': 'https://www.redbus.in/bus-tickets/bangalore-to-kadapa',
 'APSRTC_Hyderabad to Narasaraopet': 'https://www.redbus.in/bus-tickets/hyderabad-to-narsaraopet',
 'APSRTC_Chittoor (Andhra Pradesh) to Bangalore': 'https://www.redbus.in/bus-tickets/chittoor-andhra-pradesh-to-bangalore',
 'APSRTC_Visakhapatnam to Kakinada': 'https://www.redbus.in/bus-tickets/visakhapatnam-to-kakinada',
 'APSRTC_Narasaraopet to Hyd

In [221]:
driver = webdriver.Firefox()
driver.maximize_window()
df = pd.DataFrame(columns=['route_name', 'route_link', 'bus_name', 'bus_type', 'departing_time', 'duration','reaching_time', 'star_rating', 'price', 'seats_available'])

for route_name, route_link in travels_links_dict.items():
    try:
        driver.get(route_link)
        driver.maximize_window()
        time.sleep(5)

        # Wait for the 'View Buses' button
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'button')))
        driver.back()

        body = driver.find_element(By.TAG_NAME, "body")
        body.send_keys(Keys.PAGE_DOWN)

        # Click "View Buses" buttons
        view_buses_buttons = [driver.find_elements(By.CLASS_NAME, "button")]
        for button in view_buses_buttons[0]:
            if button.text == "View Buses" or button.text == "VIEW BUSES":
                print(button.text)
                button.click()
                print(button.text)

        # Scroll to load full content
        scrolling = True
        while scrolling:
            old_page_source = driver.page_source
            body.send_keys(Keys.ARROW_UP)
            new_page_source = driver.page_source
            if new_page_source == old_page_source:
                scrolling = False

        scrolling = True
        while scrolling:
            old_page_source = driver.page_source
            body.send_keys(Keys.CONTROL + Keys.END)
            time.sleep(1)
            new_page_source = driver.page_source
            if new_page_source == old_page_source:
                scrolling = False

        scrolling = True
        while scrolling:
            old_page_source = driver.page_source
            body.send_keys(Keys.ARROW_UP)
            new_page_source = driver.page_source
            if new_page_source == old_page_source:
                scrolling = False

        details_box = driver.find_elements(By.XPATH, "//div[contains(@class, 'clearfix') and contains(@class, 'row-one')]")
        bus_details = parse_bus_details(details_box=details_box,df=df,route_link=route_link,route_name=route_name)

    
    except Exception as e:
        print(f"Exception occurred for route {route_name}. Skipping to the Next Route")
        continue


Exception occurred for route APSRTC_Hyderabad to Vijayawada: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Exception occurred for route APSRTC_Vijayawada to Hyderabad: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

VIEW BUSES
HIDE BUSES
VIEW BUSES
HIDE BUSES
VIEW BUSES
HIDE BUSES
VIEW BUSES
HIDE BUSES


KeyboardInterrupt: 

In [222]:
bus_details

,route_name,route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seats_available
0,APSRTC_Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 45776,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",06:30,08h 10m,14:40,3.3,555,28 Seats
1,APSRTC_Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 5151,STAR LINER(NON-AC SLEEPER 2+1),18:00,09h 15m,03:15 (16-Jan),3.9,719,20 Seats | 4 Window
2,APSRTC_Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 5070,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",18:45,08h 30m,03:15 (16-Jan),3.8,556,27 Seats
3,APSRTC_Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 92158,INDRA(A.C. Seater),19:00,09h 00m,04:00 (16-Jan),4.0,671,21 Seats | 9 Window
4,APSRTC_Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 48821,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",19:00,09h 15m,04:15 (16-Jan),4.2,555,27 Seats
...,...,...,...,...,...,...,...,...,...,...
187,APSRTC_Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...,SRI MARUTHI TRAVELS,Non A/C Seater / Sleeper (2+1),22:30,07h 30m,06:00 (16-Jan),1.5,1111,43 Seats | 13 Window
188,APSRTC_Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...,Shree Saipooja Travels,A/C Seater/Sleeper (2+1),22:50,06h 25m,05:15 (16-Jan),2.7,833,42 Seats | 10 Window
189,APSRTC_Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...,VV Vinayak Travels,Non A/C Seater / Sleeper (2+1),21:10,08h 50m,06:00 (16-Jan),3.5,5000,41 Seats | 10 Window
190,APSRTC_Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...,SRI MARUTHI TRAVELS,Non A/C Seater / Sleeper (2+1),21:45,08h 45m,06:30 (16-Jan),1.5,2222,37 Seats | 7 Window


In [220]:
def parse_bus_details(details_box, df, route_name, route_link):
    for element in details_box:
        try:
            # Bus name
            bus_name = element.find_element("css selector", ".travels").text
            
            # Bus type
            bus_type = element.find_element("css selector", ".bus-type.f-12").text
            
            # Departure time
            departure_time = element.find_element("css selector", ".dp-time.f-19").text
            
            # Duration
            duration = element.find_element("css selector", ".dur.l-color").text
            
            # Arrival time + date
            arrival_time = element.find_element("css selector", ".bp-time.f-19").text
            try:
                next_day = element.find_element("css selector", ".next-day-dp-lbl").text
                arrival_datetime = f"{arrival_time} ({next_day})"
            except:
                arrival_datetime = arrival_time
            
            # Rating - handles both numeric and "New"
            try:
                rating_element = element.find_element("css selector", ".lh-18.rating span")
                rating = rating_element.text
            except:
                try:
                    rating = element.find_element("css selector", ".rate_count").text
                except:
                    rating = "N/A"
            
            # Price
            price = element.find_element("css selector", ".fare .f-19").text
            
            # Available seats and window seats
            try:
                seats = element.find_element("css selector", ".seat-left").text.split()[0]
            except:
                seats = "0"
                
            # Window seats - with null handling
            try:
                window_seats = element.find_element("css selector", ".window-left").text.split()[0]
                seats_info = f"{seats} Seats | {window_seats} Window"
            except:
                seats_info = f"{seats} Seats"
            
            # Create new row data
            new_row = {
                'route_name': route_name,
                'route_link': route_link,
                'bus_name': bus_name,
                'bus_type': bus_type,
                'departing_time': departure_time,
                'duration': duration,
                'reaching_time': arrival_datetime,
                'star_rating': rating,
                'price': price,
                'seats_available': seats_info
            }
            
            # Append the new row to the DataFrame
            df.loc[len(df)] = new_row
            
        except Exception as e:
            print(f"Error parsing bus details: {e}")
            continue
    
    return df

In [ ]:
bus_details

[{'bus_name': 'APSRTC - 35188',
  'bus_type': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)',
  'departure_time': '00:40',
  'duration': '06h 05m',
  'arrival_time': '06:45',
  'star_rating': '2.7',
  'price': '737',
  'available_seats': '18 Seats available'},
 {'bus_name': 'APSRTC - 3563',
  'bus_type': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)',
  'departure_time': '01:00',
  'duration': '06h 15m',
  'arrival_time': '07:15',
  'star_rating': '3.9',
  'price': '412',
  'available_seats': '24 Seats available'},
 {'bus_name': 'APSRTC - 35266',
  'bus_type': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)',
  'departure_time': '01:25',
  'duration': '06h 05m',
  'arrival_time': '07:30',
  'star_rating': '3.7',
  'price': '737',
  'available_seats': '16 Seats available'},
 {'bus_name': 'APSRTC - 3590',
  'bus_type': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)',
  'departure_time': '01:45',
  'duration': '06h 15m',
  'arrival_time': '08:00',
  'star_rating': '4.5',
  'price': '412',
  'available_seats': '31 S

In [167]:
for element in details_box:
    print(element.text)
    print('\n')

APSRTC - 35188
SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)
00:40
Shamshabad
06h 05m
06:45
Rtc bus stand
2.7
9
INR 737
18 Seats available
2


APSRTC - 3563
SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)
01:00
Central Bus Station (CBS)
06h 15m
07:15
Benz Circle
3.9
34
INR 412
24 Seats available


APSRTC - 35266
SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)
01:25
Shamshabad
06h 05m
07:30
Rtc bus stand
3.7
26
INR 737
16 Seats available


APSRTC - 3590
SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)
01:45
Central Bus Station (CBS)
06h 15m
08:00
Benz Circle
4.5
13
INR 412
31 Seats available


APSRTC - 3058
SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)
02:10
Shamshabad
06h 05m
08:15
Rtc bus stand
4.0
26
INR 737
7 Seats available


APSRTC - 3548
SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)
03:00
Central Bus Station (CBS)
06h 20m
09:20
Benz Circle
3.7
16
INR 412
32 Seats available


APSRTC - 35197
DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle)
03:10
Shamshabad
05h 50m
09:00
Rtc bus stand
4.7
62
INR 633
38 Seats available
18 Window


AP

In [159]:
print(len(bus_names))
print(len(bus_types))
print(len(departing_time))
print(len(duration))
print(len(arrival_time))
print(len(prices))
print(len(ratings_1))
print(len(ratings_2))
print(len(ratings_3))
print(len(ratings_4))
print(len(ratings_5))

421
421
421
421
421
421
0
411
411
411
0


In [146]:
for index,bus in enumerate(bus_names):
    print(f"{bus.text}")
    print(bus_types[index].text)
    print(departing_time[index].text)
    print(duration[index].text)
    print(arrival_time[index].text)
    print(ratings[index].text)

APSRTC - 35188
SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)
00:40
06h 05m
06:45
2.7
APSRTC - 3563
SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)
01:00
06h 15m
07:15
3.9
APSRTC - 35266
SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)
01:25
06h 05m
07:30
3.7
APSRTC - 3590
SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)
01:45
06h 15m
08:00
4.5
APSRTC - 3058
SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)
02:10
06h 05m
08:15
4.0
APSRTC - 3548
SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)
03:00
06h 20m
09:20
3.7
APSRTC - 35197
DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle)
03:10
05h 50m
09:00
4.7
APSRTC - 35190
DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle)
04:10
05h 55m
10:05
4.5
APSRTC - 3558
AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)
04:20
05h 25m
09:45
4.2
APSRTC - 35267
INDRA(A.C. Seater)
04:40
05h 50m
10:30
3.9
APSRTC - 3540
SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)
04:45
07h 20m
12:05
3.3
APSRTC - 3586
AMARAVATHI (VOLVO / SCANIA A.C Multi Axle)
05:10
05h 50m
11:00
4.2
APSRTC - 35196
SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)
05:50
07h 00m